In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import yaml
import os
import numpy as np
import unicodedata
import re
import json
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)

# Define function to load settings from YAML file
def load_settings(yaml_file):
    logging.info(f"Loading settings from {yaml_file}")
    with open(yaml_file, 'r') as f:
        settings = yaml.safe_load(f)
    logging.info("Settings loaded successfully")
    return settings

# Define function to load and process data from .xlsx file
def load_data(xlsx_file, settings):
    logging.info(f"Loading data from {xlsx_file}")
    df = pd.read_excel(xlsx_file)
    # Discard columns
    df = df.drop(settings['xlsx_processing']['discard_columns'], axis=1)
    # Remove accentuation
    if settings['xlsx_processing']['remove_accentuation']:
        df = df.applymap(lambda x: unicodedata.normalize('NFKD', str(x)).encode('ASCII', 'ignore').decode('utf-8') if isinstance(x, str) else x)
    # Convert dataframe to JSON
    json_data = df.to_json(orient='records')
    logging.info("Data loaded and processed successfully")
    return json_data

# Define function to generate heatmap
def generate_heatmap(json_data, competence, assessors, settings):
    logging.info(f"Generating heatmap for {competence}")
    # Filter data for current competence and assessors
    data = [d for d in json_data if d['competencia'] == competence and d['avaliador'] in assessors]
    # Create dataframe with single assessors as index and objective questions as columns
    df = pd.DataFrame(data)
    df = df.pivot(index='assessor', columns='question', values='score')
    # Generate heatmap
    sns.heatmap(df, **settings['heatmap_formatting'])
    plt.title(settings['heatmap_formatting']['title_template'].format(competence=competence, pessoa=df.index[0]))
    plt.show()
    logging.info("Heatmap generated successfully")

# Ask user for name of .xlsx file to be processed
xlsx_file = input("Enter the name of the .xlsx file to be processed: ")
xlsx_file = f"{xlsx_file}.xlsx"

# Load settings from YAML file
settings = load_settings("competencias.yaml")

# Load and process data from .xlsx file
json_data = load_data(xlsx_file, settings)

# Generate heatmap for each competence
for competence in settings['competencias']:
    assessors = competence['avaliadores']
    generate_heatmap(json_data, competence['nome'], assessors, settings)